In [ ]:
from grid.eval import *

In [ ]:
boardsize = 5
n_workers = 4

snaps = snapshots(boardsize)
snaps = pd.concat([snaps, parameters(snaps)], 1)
snaps['nickname'] = snaps.run.str.extract('.* (.*)', expand=False) + '.' + snaps.idx.astype(str)
snaps['params'] = params(snaps)
snaps = snaps.set_index('nickname')

games, wins = load(boardsize, snaps.index)

soln = None
futures = {}
solver = None
with ProcessPoolExecutor(n_workers, initializer=init) as pool:
    while True:
        if solver is None:
            log.info('Submitting solve taask')
            solver = pool.submit(activelo.solve, games, wins, soln=soln)
        elif solver.done():
            try:
                log.info('Got solve task')
                soln = solver.result()
            except InManifoldError:
                soln = None
                log.warning('Got a manifold error; throwing soln out')
            finally:
                solver = None

        for key, future in list(futures.items()):
            if future.done():
                log.info('Got eval task result')
                results = future.result()
                games, wins = update(games, wins, results)
                del futures[key]
                save(boardsize, games, wins)

        while len(futures) < n_workers - 1:
            if soln is None:
                sugg = tuple(np.random.choice(games.index, (2,)))
            else:
                sugg = suggest(soln)
            
            log.info('Submitting eval task')
            futures[sugg] = pool.submit(evaluate, *sugg)
    
        if soln is not None:
            report(soln, games, futures)
            _, σ = arena.analysis.difference(soln, soln.μ.idxmin())
            if σ.pow(2).mean()**.5 < .01:
                break
        
snaps['μ'], snaps['σ'] = arena.analysis.difference(soln, soln.μ.idxmin())

In [ ]:
%timeit -n1 -r1 activelo.solve(games, wins, soln=soln)

In [ ]:
snaps['μ'], snaps['σ'] = arena.analysis.difference(soln, soln.μ.idxmax())

In [ ]:
plot(snaps)